In [178]:
from promptolution.llms import APILLM
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Normalize
import numpy as np
from seaborn import heatmap

In [179]:
token = open("../deepinfratoken.txt", "r").read()
meta_llm = APILLM("meta-llama/Meta-Llama-3-8B-Instruct", token)

In [180]:
similarity_model = SentenceTransformer(
    "NovaSearch/stella_en_400M_v5", trust_remote_code=True, device="cuda"
)

Some weights of the model checkpoint at NovaSearch/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [181]:
similarity_model = similarity_model.append(Normalize())

In [182]:
def eval_similarities(embs_a, embs_b, similarity_model) -> np.array:
    similarities = np.dot(embs_a, embs_b.T)
    # similarities = np.triu(similarities, 1)

    return similarities

# Sample Prompts

In [215]:
base_prompts = [
    "Your task is to identify the primary topic of the news artical and choose from World, Sports, Business and Tech.",
    "Your job is to determine whether a news article belongs to the World, Sports, Business, or Tech category based on its primary theme.",
    "You will be given a news article and asked to classify it as World, Sports, Business and Tech, depending on its main topic.",
    "Your responsibility is to assign a news article to one of four categories: World, Sports, Business, or Tech, based on its main idea.",
    "Categorize the news article into one of four categories: World, Sports, Business, or Tech, based on its content.",
]

# EvoPromptDE Mutation Prompt

=> would not allow us to seperate mutation from cross over

In [184]:
meta_prompt = """Please follow the instruction step-by-step to generate a better prompt.
Identifying the different parts between Prompt 1 and Prompt 2:
Prompt 1: Your task is to classify the comment as one of the following categories: terrible, bad, okay, good, great.
Prompt 2: In this task, you are given sentences from movie reviews. The task is to classify a sentence as one of the following categories: terrible, bad, okay, good, great.
Different parts:
"Your task is to classify the comment" vs "In this task, you are given sentences from movie reviews. The task is to classify a sentence"
"comment" vs "sentences from movie reviews"

2. Randomly mutate the different parts:
"Your task is to classify the comment" -> "The objective is to categorize the statement"
"comment" -> "phrases in movie reviews"

3. Crossover the different parts with the following Prompt 3 and generate a final prompt bracketed with <prompt> and </prompt>:
Prompt 3: You are a sentiment classifier. To do this, you must first understand the meaning of the sentence and any relevant context. And then you should classify it as one of the following categories: terrible, bad, okay, good, great.

Final Prompt: <prompt>As a sentiment classifier, analyze phrases in movie reviews and categorize them into one of the following categories: terrible, bad, okay, good, great, while considering the meaning and relevant context.</prompt>

Please follow the instruction step-by-step to generate a better prompt.
1. Identify the different parts between the Prompt 1 and Prompt 2:
Prompt 1: <prompt1>
Prompt 2: <prompt2>
2. Randomly mutate the different parts
3. Crossover the different parts with the following Prompt 3 and generate a final prompt bracketed with <prompt> and </prompt>:
Prompt 3: <prompt0>

1."""

In [185]:
new_prompts = []
for prompt_0 in base_prompts:
    for prompt_1 in base_prompts:
        for prompt_2 in base_prompts:
            mp = (
                meta_prompt.replace("<prompt1>", prompt_1)
                .replace("<prompt2>", prompt_2)
                .replace("<prompt0>", prompt_0)
            )
            new_prompt = meta_llm.get_response([mp])[0]
            new_prompt = new_prompt.split("<prompt>")[-1].split("</prompt>")[0]
            new_prompts.append(new_prompt)

In [186]:
print(new_prompts)

["Let's follow the instructions step-by-step to generate a better prompt.\n\n1. Identify the different parts between Prompt 1 and Prompt 2:\n\nPrompt 1: Please perform Question Classification task. Given the question, assign a label from ['Description', 'Entity', 'Expression', 'Human', 'Location', 'Number']. Return label only without any other text\n\nPrompt 2: Please perform Question Classification task. Given the question, assign a label from ['Description', 'Entity', 'Expression', 'Human', 'Location', 'Number']. Return label only without any other text\n\nDifferent parts:\n\n* None (both prompts are identical)\n\nSince the prompts are identical, there is no need to mutate or crossover parts. Therefore, we can proceed to the next step.\n\n(Note: If you'd like to introduce minor variations, you can try replacing small phrases or words, but since the prompts are identical, it's not necessary in this case.)", ' and ', 'Let\'s follow the instructions step-by-step to generate a better pro

In [187]:
embs_old = similarity_model.encode(base_prompts)
embs_new = similarity_model.encode(new_prompts)
sims_old_new = eval_similarities(embs_old, embs_new, similarity_model)
sims_new_new = eval_similarities(embs_new, embs_new, similarity_model)

In [188]:
print(sims_old_new.mean())
print(sims_new_new.mean())
print(sims_new_new.std())

0.69002473
0.62920034
0.19569868


# EvoPrompt GA

In [216]:
meta_prompt = """Please follow the instruction step-by-step to generate a better prompt.
1. Crossover the following prompts and generate a new prompt:
Prompt 1: Rewrite the input text into simpler text.
Prompt 2: Rewrite my complex sentence in simpler terms, but keep the meaning.
2. Mutate the prompt generated in Step 1 and generate a final prompt bracketed with <prompt> and </prompt>.

1. Crossover Prompt: Rewrite the complex text into simpler text while keeping its meaning.
2. <prompt>Transform the provided text into simpler language, maintaining its essence.</prompt>

Please follow the instruction step-by-step to generate a better prompt.
1. Crossover the following prompts and generate a new prompt:
Prompt 1: <prompt1>
Prompt 2: <prompt2>
2. Mutate the prompt generated in Step 1 and generate a final prompt bracketed with <prompt> and </prompt>.

1."""

In [217]:
new_prompts = []

for _ in range(2):
    for prompt_1 in base_prompts:
        for prompt_2 in base_prompts:
            mp = meta_prompt.replace("<prompt1>", prompt_1).replace("<prompt2>", prompt_2)
            new_prompt = meta_llm.get_response([mp])[0]
            new_prompt = new_prompt.split("<prompt>")[-1].split("</prompt>")[0]
            new_prompts.append(new_prompt)

In [218]:
embs_old = similarity_model.encode(base_prompts)
embs_new = similarity_model.encode(new_prompts)
sims_old_new = eval_similarities(embs_old, embs_new, similarity_model)
sims_new_new = eval_similarities(embs_new, embs_new, similarity_model)

In [219]:
print(new_prompts)

[' and ', 'Classify news articles by identifying their primary theme and assigning them to one of four categories: World, Sports, Business, or Tech.', 'Determine the primary topic of a news article and classify it as World, Sports, Business, or Tech by identifying its central theme.', "Please provide the second prompt, and I'll be happy to help you generate a new prompt by crossing over the two and then mutating the result.\n\nPlease go ahead and provide the second prompt.", 'Determine the main focus of the news article by sorting it into one of the predefined categories – either World, Sports, Business, or Tech – using the content as a guide.', 'Classify news articles by their primary theme, grouping them into World, Sports, Business, or Tech categories.', 'Since the crossover of the two prompts results in identical prompts, I will proceed to mutate the prompt to generate a new one.\n\nMutated Prompt:\n<eventually-categorize-articles>Classify news articles into World, Sports, Business

In [220]:
print(sims_old_new.mean())
print(sims_new_new.mean())
print(sims_new_new.std())

0.84156543
0.7692937
0.23206602


# Prompt Variation template

In [221]:
meta_prompt = """Generate a single variation of the following instruction while keeping the semantic meaning.
Generate the variation starting with <prompt> and ending with </prompt> tags.

Input: <prev_prompt>

Output:"""

In [222]:
new_prompts = []
for _ in range(4):
    for prompt in base_prompts:
        mp = meta_prompt.replace("<prev_prompt>", prompt)
        new_prompt = meta_llm.get_response([mp])[0]
        new_prompt = new_prompt.split("<prompt>")[-1].split("</prompt>")[0]
        new_prompts.append(new_prompt)

In [223]:
print(new_prompts)

['Categorize the news article under the topic that best represents its primary theme, choosing from World, Sports, Business, and Technology.', 'Classify a news article into one of four categories - World, Sports, Business, or Tech - based on its main topic of discussion.', 'You will be provided with a news article and required to categorize it as World, Sports, Business, or Tech, based on its primary subject matter.', "You'll categorize news articles into one of four main areas: World News, Sports News, Business Insights, or Tech Updates, depending on the primary topic of the article.", 'Classify the news article into one of the following categories: Global, Athletics, Finance, or High-Tech, according to its subject matter.', 'Choose the primary topic of the news article from the options World, Sports, Business, or Tech.', " Classify a news article into one of four categories: World, Sports, Business, or Tech, based on the article's primary theme.", 'You will be presented with a news a

In [224]:
embs_old = similarity_model.encode(base_prompts)
embs_new = similarity_model.encode(new_prompts)
sims_old_new = eval_similarities(embs_old, embs_new, similarity_model)
sims_new_new = eval_similarities(embs_new, embs_new, similarity_model)

In [225]:
print(sims_old_new.mean())
print(sims_new_new.mean())
print(sims_new_new.std())

0.882621
0.8461745
0.096574746


# Basic really random prompt

In [226]:
meta_prompt = """
Improve the prompt and return the result between <prompt> and </prompt>:

<instruction>
"""

In [227]:
new_prompts = []
for _ in range(4):
    for prompt in base_prompts:
        mp = meta_prompt.replace("<instruction>", prompt)
        new_prompt = meta_llm.get_response([mp])[0]
        new_prompt = new_prompt.split("<prompt>")[-1].split("</prompt>")[0]
        new_prompts.append(new_prompt)

In [228]:
print(new_prompts)

['Classify the primary topic of the news article into one of the following categories: World, Sports, Business, or Tech.', 'Randomly generate a news article paragraph, then ask me to categorize it into one of four categories: World, Sports, Business, or Tech, based on its primary theme. The article should be realistic and descriptive enough to require some thought to categorize correctly.', '<Prompt>Classify a given news article into one of the following categories: World, Sports, Business, and Tech, based on its main topic. Please read the article and identify the primary subject matter and categorize it accordingly.</Prompt>', 'Classify a news article into one of four categories: World, Sports, Business, or Tech, based on its main idea. Provide a brief summary of the article and justify your categorization.', "Categorize the provided news article into one of four categories: World, Sports, Business, or Tech, based on its content. Please analyze the article's headlines, keywords, and 

In [229]:
embs_old = similarity_model.encode(base_prompts)
embs_new = similarity_model.encode(new_prompts)
sims_old_new = eval_similarities(embs_old, embs_new, similarity_model)
sims_new_new = eval_similarities(embs_new, embs_new, similarity_model)

In [230]:
print(sims_old_new.mean())
print(sims_new_new.mean())
print(sims_new_new.std())

0.9455951
0.94949603
0.033495728


# A new idea

In [231]:
# prompt taking a single prompt and generating a new one
meta_prompt = """From the given prompt, generate a new prompt that is similar in meaning. Return the new prompt between <prompt> and </prompt> tags.

<instruction>
"""

In [232]:
new_prompts = []
for _ in range(4):
    for prompt in base_prompts:
        mp = meta_prompt.replace("<instruction>", prompt)
        new_prompt = meta_llm.get_response([mp])[0]
        new_prompt = new_prompt.split("<prompt>")[-1].split("</prompt>")[0]
        new_prompts.append(new_prompt)

In [233]:
print(new_prompts)

[' gibberish\n\n<Prompt>The task is to determine the core subject of the news report and categorize it as Entertainment, Politics, Health, or Culture.</Prompt>', 'Automatically categorize a given news article into one of four categories (World, Sports, Business, or Tech) based on its main subject matter.', 'Classify a given news article into one of the following categories: General, Politics, Entertainment, or Science, based on its primary subject matter.', 'Classify a news headline into one of four categories: Global News, Athletic Events, Financial Updates, or Digital Developments, based on its primary focus.', 'Classify the news headline into one of four categories: Global News, Sports Update, Financial Report, or Technological Breakthrough, based on its theme.', 'Classify the main theme of the article as World, Sports, Business, or Technology.', "Classify a news article into one of the following categories: News, Entertainment, Politics, or Sciences, based on the article's primary 

In [234]:
embs_old = similarity_model.encode(base_prompts)
embs_new = similarity_model.encode(new_prompts)
sims_old_new = eval_similarities(embs_old, embs_new, similarity_model)
sims_new_new = eval_similarities(embs_new, embs_new, similarity_model)

In [235]:
print(sims_old_new.mean())
print(sims_new_new.mean())
print(sims_new_new.std())

0.79380965
0.8067877
0.092519104


# Human Gradient Descent

In [236]:
meta_prompt = """Please do the following for this task: The dataset contains news articles categorized into four classes: World, Sports, Business, and Tech. The task is to classify each news article into one of the four categories. The class mentioned first in the response of the LLM will be the prediction.
Rephrase the following instruction while preserving its core meaning, while substantially differing in linguistcal style.
Return the new prompt between <prompt> and </prompt> tags.

<instruction>
"""

In [237]:
new_prompts = []
for _ in range(4):
    for prompt in base_prompts:
        mp = meta_prompt.replace("<instruction>", prompt)
        out = meta_llm.get_response([mp])[0]
        new_prompt = out.split("<prompt>")[-1].split("</prompt>")[0]
        new_prompts.append(new_prompt)

In [238]:
print(new_prompts)

["\nDelve into the essence of each news article and pinpoint its dominant theme. This involves identifying the core subject matter and assigning it to one of four categories: World, Sports, Business, or Tech. The primary classification will be based on the subject mentioned first in the LLM's output.", '\nClassify each news article by assigning one of the four fundamental categorizations: World, Sports, Business, or Tech, relying on the dominant topic or theme presented in the article. The categorization you propose will be the one that initially comes to mind for the LLM.', 'Prepare to categorize a news item into one of four designated realms: Worldly affairs, Sporting highlights, Business updates, or Tech tidbits. Upon receiving the article, pls identify the primary focus and assign the most fitting category based on that.', "Classify the given news article into one of four broad categories: 'Global News', 'Athletes' Activations', 'Market Movement', or 'Technological Trends', choosin

In [239]:
embs_old = similarity_model.encode(base_prompts)
embs_new = similarity_model.encode(new_prompts)
sims_old_new = eval_similarities(embs_old, embs_new, similarity_model)
sims_new_new = eval_similarities(embs_new, embs_new, similarity_model)

In [240]:
print(sims_old_new.mean())
print(sims_new_new.mean())
print(sims_new_new.std())

0.80516815
0.7795717
0.112174466
